In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter

In [ ]:
plt.rcParams["figure.figsize"] = (16, 3)
plt.rcParams["axes.grid"] = True

In [ ]:
ds = pd.read_csv("../csv/session.csv", delimiter=';', parse_dates=['timestamp'], index_col=0)
dg = pd.read_csv("../csv/global_returns.csv", delimiter=';', parse_dates=['timestamp'], index_col=0)
dt = pd.read_csv("../csv/processing_timings.csv", delimiter=';', parse_dates=['timestamp'])
di = pd.read_csv("../csv/issues.csv", delimiter=';', parse_dates=['timestamp'], index_col=0)


# Session data

In [ ]:
def describe(frames, types, labels):
    """
    abbreaviated version of pd.describe in horizontal layout
    
    needs 3 *size-matched* lists
    
    :param frames: a list of frames to describe
    :param types: list of event types to describe
    :param labels: labels for values of each event types
    """
    storage = []
    
    for frame, type_n, label in zip(frames, types, labels):
        
        description = frame[frame.type == type_n].describe()
        description.rename(columns={'value': label}, inplace=True)
        storage.append(description)
        
    return pd.concat(storage, axis=1).loc[['mean', 'min', 'max', 'std', 'count']].transpose()

In [ ]:
print(f"ALS start time: {str(ds.index[0])}")
print(f"ALS stop time : {str(ds.index[-1])}")

# Alignment

### translations

In [ ]:
ax = ds[ds.type == "x_trans"].plot(title="Image translations",
                                             sharex=True,
                                             y='value',
                                             label="x",
                                             ylabel="px",
                                             figsize=(16, 2))

ds[ds.type == "y_trans"].plot( y='value', label="y", ax=ax, xlabel="")



plt.show()

all_trans = [ds[(ds.type == "x_trans")], ds[(ds.type == "y_trans")]]

describe(all_trans, ['x_trans', 'y_trans'], ['translation x (px)', 'translation y (px)'])

In [ ]:
import math
rot=ds[ds.type == 'rotation'].copy()
rot['value'] = rot['value'].apply(lambda v:  math.degrees(v))


rot.plot(title="Image rotations",
    y='value', legend=False, ylabel="degrees", xlabel='', figsize=(16, 2))
plt.show()

describe([rot], ['rotation'], ['rotation (°)'])

In [ ]:
scale = ds[ds.type == 'scale']
scale.plot(y="value", legend=False, figsize=(16, 2), title="Image scalings")
plt.show()

describe([scale], ['scale'], ['scale'])


In [ ]:
rat=ds[ds.type == 'ratio'].copy()
rat['value'] = rat['value'].apply(lambda v:  v * 100)

rat.plot(
    kind="bar", y='value', label='ratio',
    ylabel="%",
    title="match detection surface ratio",
    xticks=[], xlabel="", legend=False, figsize=(16,1), ylim=(0, 100),
    grid=False
)
plt.show()

In [ ]:
matches = ds[ds.type == 'matches']
ax = matches.plot(
    y='value',
    title="frame match count vs required minimum",
    ylim=(0,ds[ds.type == 'matches'].value.max() + 5),
    xlabel="",
    label="matches",
    figsize=(16, 2)
)

ds[ds.type == 'req_matches'].plot(ax=ax, y='value', label='required', xlabel='')

plt.show()

describe([matches], ['matches'], ['matches'])

In [ ]:
ds[ds.type == 'align'].plot(
    kind="bar",
    grid=False,
    xticks=[], xlabel="",
    legend=False,
    figsize=(16,1),
    title="frame compliance to min. required match count. 1: Accepted, 0: Rejected"
)
plt.show()

In [ ]:
ds[ds.type == 'align'].value.transform(lambda x: "accepted" if x == 1. else "rejected").value_counts()

In [ ]:
data = ds[ds.type == 'align'].value.transform(lambda x: "accepted" if x == 1. else "rejected").value_counts()
data.plot(
    kind="pie",
    autopct='%1.1f%%',
    title="accepted/rejected frames distribution",
    ylabel="", explode=(([0.1 for x in range(len(data))])), figsize=(6,6)
)
plt.show()

# Issues

## Global

In [ ]:
pd.DataFrame(di.level.value_counts())

In [ ]:
di.level.hist(figsize=(2,3))
plt.show()

## Errors

In [ ]:
pd.options.display.max_colwidth = 256
pd.options.display.max_rows = 1024
di[di.level == "ERROR"].head()

## Warnings

In [ ]:
di[di.level == "WARNING"].head()

# Queues

In [ ]:
def plot_queue_size(data, title):
    
    data.plot(kind='bar', title=title,
              xlabel="",
              figsize=(16, 2),
              legend=False, xticks=[])
    plt.show()

In [ ]:
plot_queue_size(ds[ds.type == 'q_pre'], "Pre-processing queue size")

In [ ]:
plot_queue_size(ds[ds.type == 'q_stack'], "Stacking queue size")

In [ ]:
plot_queue_size(ds[ds.type == 'q_post'], "Post-processing queue size")

In [ ]:
plot_queue_size(ds[ds.type == 'q_save'], "Save queue size")

In [ ]:
data = ds[ds.type.str.startswith('q_')]
data = data.replace(to_replace={
    "q_pre": "Pre-processing",
    "q_stack": "Stacking",
    "q_post": "Post-processing",
    "q_save": "Save",
    })

plt.figure(figsize=(16, 6))
plt.grid(False)
ax = sns.barplot(data = data, x=data.index, y='value', hue='type')
ax.set(ylabel="", title="Combined queue sizes")
ax.set(xticks=[])
plt.show()



# Timings

In [ ]:
total = ds[ds.type == 'frm_total']
total.plot(title="Total frame processing time",
                                ylabel="second", figsize=(16, 6), legend=False)
plt.show()
describe([total], ['frm_total'], ['Total frame processing time'])

In [ ]:
def plot_timings(name_dict, title):

    pre_data = dt[dt['name'].isin(name_dict.keys())].copy()
    
    for k, v in name_dict.items():
        pre_data.loc[pre_data['name'] == k, 'name'] = v
        
    pre_data = pre_data.rename(columns={'name': 'process'})
    
    sns.lineplot(data=pre_data, x='timestamp', y='elapsed', hue='process').set(title=title, ylabel='ms')
    plt.plot()
    return pre_data

    

In [ ]:
prepro_name_dict = {
    'HotPixelRemover.process_image()': 'H.P. Remove',
    'RemoveDark.process_image()': 'Dark remove',
    'Debayer.process_image()': 'Debayer',
    'Standardize.process_image()': 'Nomalize',
    'read_disk_image()': 'Load'
}

pre_data = plot_timings(prepro_name_dict, "Pre-processing time")



In [ ]:
pre_data.groupby('process').describe().iloc[:, [1,3,7,0,2]]

In [ ]:
stack_name_dict = {
    "Stacker._find_transformation()": "Find transf.",
    "Stacker._apply_transformation()": "Apply Transf.",
    "Stacker._stack_image()": "Stack"
}

st_data = plot_timings(stack_name_dict, "Stacking time")

In [ ]:
st_data.groupby('process').describe().iloc[:, [1,3,7,0,2]]

In [ ]:
old_new_pp_names_dict = {
    'AutoStretch.process_image()': 'Stretch',
    'Levels.process_image()': 'Levels',
    'ColorBalance.process_image()': 'Col. Balance',
    'ImageSaver._save_image()': 'Save'
}

po_data = plot_timings(old_new_pp_names_dict, "Post-process timing")

In [ ]:
po_data.groupby('process').describe().iloc[:, [1,3,7,0,2]]